# Churn Model Training + Deployment Pipeline

In [1]:
import mlrun
import pandas as pd

### Setup Project

In [2]:
project = mlrun.get_or_create_project(
    "churn-agents-webinar",
    parameters={
        "base_image" : "mlrun/mlrun:1.7.2",
        "requirements_file" : "requirements-churn-pipeline.txt",
        "force_build" : False,
        "source" : "v3io:///bigdata/churn.zip", # enterprise
        # "source" : "s3://mlrun/churn.zip" # open source
    }
)

Project Source: v3io:///bigdata/churn.zip
Exporting project as zip archive to v3io:///bigdata/churn.zip...
> 2025-01-22 17:13:06,425 [info] Project loaded successfully: {"project_name":"churn-agents-webinar"}


### Log Dataset

In [45]:
import pandas as pd

dataset = project.log_dataset(key="churn", df=pd.read_csv("data/churn_dataset.csv"))
dataset.uri

'store://datasets/churn-agents-webinar/churn#0@a14f19aadf033e34631adad4a8488b912d14e98d'

### Run Pipeline

In [46]:
from config import MainConfig

In [47]:
arguments = MainConfig(source_url = dataset.uri).dict()
arguments

{'source_url': 'store://datasets/churn-agents-webinar/churn#0@a14f19aadf033e34631adad4a8488b912d14e98d',
 'label_column': 'churn',
 'allow_validation_failure': True,
 'test_size': 0.2,
 'model_name': 'churn_model',
 'sentiment_model': 'cardiffnlp/twitter-roberta-base-sentiment-latest',
 'text_column': 'chat_log',
 'sentiment_column': 'sentiment_label',
 'ordinal_columns': ['international_plan', 'voice_mail_plan'],
 'drop_columns': ['area_code', 'chat_log', 'state']}

In [48]:
train_run_id = project.run(
    name="main",
    arguments=arguments,
    dirty=True,
    watch=True
)

> 2025-01-21 19:45:39,139 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2025-01-21 19:45:40,980 [info] Pipeline submitted successfully: {"id":"1f9e7a2e-4fdc-4250-92c5-9f7721844462","pipeline_name":"churn-agents-webinar-main 2025-01-21 19-45-40"}
> 2025-01-21 19:45:40,981 [info] Pipeline run id=1f9e7a2e-4fdc-4250-92c5-9f7721844462, check UI for progress


> 2025-01-21 19:45:41,050 [info] Started run workflow churn-agents-webinar-main with run id = '1f9e7a2e-4fdc-4250-92c5-9f7721844462' by kfp engine
> 2025-01-21 19:45:41,051 [info] Waiting for pipeline run completion: {"project":"churn-agents-webinar","run_id":"1f9e7a2e-4fdc-4250-92c5-9f7721844462"}


uid,start,state,kind,name,parameters,results
...845c07be,Jan 21 19:54:13,completed,run,validate-validate-model,model_path=store://models/churn-agents-webinar/churn_model#1:latest@1f9e7a2e-4fdc-4250-92c5-9f7721844462label_column=churnallow_validation_failure=True,passed_suite=False
...f4cd991b,Jan 21 19:53:00,completed,run,train,label_column=churnmodel_name=churn_model,best_iteration=4accuracy=0.9583333333333334f1_score=0.8571428571428571precision_score=0.868421052631579recall_score=0.8461538461538461model_uri=store://models/churn-agents-webinar/churn_model#1:latest@1f9e7a2e-4fdc-4250-92c5-9f7721844462
...e31a4689,Jan 21 19:52:28,completed,run,validate-validate-train-test-split,label_column=churnallow_validation_failure=True,passed_suite=True
...e3092664,Jan 21 19:52:02,completed,run,data-process-data,"label_column=churntest_size=0.2ordinal_columns=['international_plan', 'voice_mail_plan']drop_columns=['area_code', 'chat_log', 'state']sentiment_column=sentiment_label",
...b931b394,Jan 21 19:51:05,completed,run,validate-validate-data-integrity,label_column=churnallow_validation_failure=True,passed_suite=False
...20c5a20b,Jan 21 19:46:15,completed,run,data-sentiment-analysis,sentiment_model=cardiffnlp/twitter-roberta-base-sentiment-latesttext_column=chat_log,
...894378d5,Jan 21 19:45:48,completed,run,data-get-data,,


### Test Model Endpoint

In [18]:
serving_fn = project.get_function("serving")

In [19]:
test = mlrun.get_dataitem("store://datasets/churn-agents-webinar/data-process-data_test#0:latest").as_df()
test = test.drop("churn", axis=1)
data = test.head(5).to_dict(orient="split")["data"]

In [22]:
%%time
serving_fn.invoke(
    "/v2/models/churn_model/predict",
    body={
        "inputs": data
    }
)

> 2025-01-23 17:44:31,325 [info] Invoking function: {"method":"POST","path":"http://nuclio-churn-agents-webinar-serving.default-tenant.svc.cluster.local:8080/v2/models/churn_model/predict"}
CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 38 ms


{'id': '62d07bd5-6a18-444a-a5a6-bb9063a67045',
 'model_name': 'churn_model',
 'outputs': [0.007601190476190476,
  0.0029821428571428573,
  0.005812500000000001,
  0.009351190476190476,
  0.004410714285714285],
 'timestamp': '2025-01-23 17:44:31.334132+00:00',
 'model_version': 'latest'}